# 2.4 Data Transformation

So far we've covered basic exploration of the data. But often we want to see it in a different shape. Aggregations are one way of doing this, reshaping and pivoting are others.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data/imdb_clean.csv")

# Show the first few rows of data with df.head()
df.head()

## Pivoting Data

Suppose we want to know the number of titles released per year, by type. Lets pivot the data into a format that shows that with df.pivot_table()

### A note on `pivot` and `pivot_table`
Pandas dataframes have two pivot methods. `pivot()` is used where the data is only being reshaped and no aggregation is being performed. `pivot_table()` lets us specify an aggregation, and works more like a pivot table in excel for example.


In [ ]:
types_by_year = df.pivot_table(
    index='startYear', # Specify what you want in the rows here - you can use a list of columns too.
    columns='titleType', # Specify what you want in the columns here - you can use a list of columns too.
    aggfunc='count', # Which aggregation you want to use.
    values = 'id' # What column you want to aggregate
)

display(types_by_year)

## Aggregating Data

Aggregating data in pandas is most often done with `groupby`. This creates a 'grouper' object that you can reuse to generate multiple aggregations.

Lets group our datafame by year and titleType, and then find some other statistics.

In [ ]:
grouper = df.groupby(['startYear','titleType'])

We can pull columns we'd like to aggregate from the groupby and aggregate them. Lets get the average of all the ratings, and the total number of votes.

In [ ]:
ratings_by_year_type = grouper.agg({'averageRating':'mean', 'numVotes': 'sum'})
ratings_by_year_type

You might notice startYear and titleType in bold above. This is because they are now both part of the index - when we use groupby pandas creates a new index using the columns youve grouped by. Where there are multiple columns pandas creates a hierarchical index called a multiindex. We can access values from it like this.

In [ ]:
# Get ratings for movies in 2001
ratings_by_year_type.loc[(2001,'movie')].round(1) # Round to the nearest 1 DP.

If we want to work with the index columns as pandas series objects again, we can use reset_index() to move these columns back into the dataframe, and generate a simple numeric index.

In [ ]:
ratings_by_year_type.reset_index()

## Joining Data

We can join data in pandas in similar ways to SQL. Datarframes can be joined in a similar fashion to a sql join with pd.merge(). Dataframes can also be unioned with `pd.concat()`. `pd.concat` can also be used to join dataframes that share common index values by changing the axis it operates on.

Lets join the average rating for the year back to our original dataframe and calculate the difference from the mean.

In [ ]:
joined_df = pd.merge(
    df,
    ratings_by_year_type,
    left_on=['startYear','titleType'], # We want to use these columns from the original dataframe.
    right_index=True, # On our aggregate dataframe, these columns form our index.
    suffixes=["","Overall"] # We can add a suffix to names on each side where they conflict (optional)
    
)

# conflicts on column_names 
joined_df['averageRatingDifference'] = joined_df['averageRatingOverall'] - joined_df['averageRating']
joined_df

In [ ]:
# Concatenating data works like this.

movies_2020 = df.loc[(df['startYear'] == 2020) & (df['titleType'] =='movie')]
video_games_2000 = df.loc[(df['startYear'] == 2000) & (df['titleType'] =='videoGame')]
short_films_1990 = df.loc[(df['startYear'] == 1990) & (df['titleType'] =='short')]

# Concatenate the data together
combined = pd.concat([movies_2020, video_games_2000, short_films_1990])

# View a few rows sorted by title.
combined.sort_values(by='primaryTitle').head(20)

## Excercises

Using the IMDB movie dataset.

- Create a pivot table showing the average  of all the episode reviews for Cheers, Friends, and Seinfeld by season.
- Generate a summary of the number of moview with "War" in the genre column by decade.
- Generate a summary of the number of moview as a proportion of all movies by decade.

In [ ]:
# Cheers, Friends, and Seinfeld.
shows = ['Cheers','Friends','Seinfeld']

shows_df = df.loc[df['parentTitle'].isin(shows)]

shows_df.pivot_table(index='parentTitle', columns='seasonNumber', values='averageRating', aggfunc='mean')


In [ ]:

# War movies by decade.

# Create a new dataframe with just our war movies.
war_movies = df['genres'].fillna('').str.contains('War') & (df['titleType'] == 'movie')
war_movies_df = df.loc[war_movies].copy()

# Calculate the number per decade.
war_movies_by_decade = war_movies_df.groupby('decade').size()
war_movies_by_decade

In [ ]:
# What about all movies?
movies = df['titleType'] == 'movie'
movies_by_decade = df.loc[movies].groupby('decade').size()
movies_by_decade

In [ ]:
# We can put  these in a dataframe by concatenating them together.
compare_df = pd.concat([war_movies_by_decade, movies_by_decade], axis=1)
compare_df.columns = ['warMovies', 'allMovies']

compare_df['percentWarMovies'] = (
    (compare_df['warMovies'] / compare_df['allMovies'] * 100)
    .round(1) # Round to 0.1%
    .fillna(0) # Fill na with zero.
)
compare_df